## Imports

In [71]:
import cv2
import os
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
import tensorflow as tf
import pickle

## Mount Drive

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Read Data 

In [76]:
def load_data(data_path):
    data = []
    fake_labels = []
    for img in os.listdir(data_path):
        img_array = cv2.imread(os.path.join(data_path, img))
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        img_array = cv2.resize(img_array, (224, 224))
        data.append(img_array)
        fake_labels.append("0")
    return data , fake_labels

# Load data
data , fake_labels = load_data("/content/drive/MyDrive/Test")
data = np.array(data)
fake_labels = np.array(fake_labels)

## Save Resized Data

In [77]:
# from pathlib import Path
# import imageio


# def save_imgs(path:Path, data):
#     for i in range(len(data)):
#         imageio.imsave( str( path/(str(i)+'.jpg') ), data[i])
# os.makedirs("ResizedTestset/0")
# save_imgs(Path('./ResizedTestset/0'), data)

## Read Resized Data for ResNet50

In [78]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
TestData = train_datagen.flow(data,fake_labels,shuffle=False)

## Connect with the model in the drive

In [79]:
! gdown "1VQ-Ruf9ZVCZuSnGtkAYR0BZix7SfzVXa"

Downloading...
From: https://drive.google.com/uc?id=1VQ-Ruf9ZVCZuSnGtkAYR0BZix7SfzVXa
To: /content/ResNet50_final_model.h5
100% 94.7M/94.7M [00:00<00:00, 212MB/s]


## Loaing ResNet50 Model

In [80]:
model = load_model("/content/ResNet50_final_model.h5")

## Get Test Data in numpy array

In [81]:
import numpy as np

test_data = []

TestData.reset()  

for _ in range(len(TestData)):
    batch_data,_ = TestData.next()
    test_data.append(batch_data)

test_data = np.concatenate(test_data)

## Test Model

In [82]:
y_pred = model.predict(test_data)

1/1 [==============================] - 6s 6s/step


## Save results

In [83]:
y_pred = np.argmax(y_pred, axis=1)
np.savetxt('Team 8.txt', y_pred, delimiter='\n' ,fmt='%.0f')